In [1]:
import pandas as pd
import numpy as np
import pickle


import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
import shutil
from d2v_recommender import *
from config import config

from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import roc_auc

import sklearn

In [2]:
d2v = D2V_Recommender()
data_dict = d2v.load_data_dict(config.data_dict_path)

In [3]:
data_dict["X_train"].shape, data_dict["y_train"].shape, data_dict["X_test"].shape, data_dict["y_test"].shape

((132919, 50), (132919,), (13334, 50), (13334,))

In [4]:
X_train = data_dict["X_train"]
X_test = data_dict["X_test"]
y_train = data_dict["y_train"]
y_test = data_dict["y_test"]

In [5]:
X_train.shape

(132919, 50)

In [6]:
# Auto-ML

shutil.rmtree(config.output_automl_path, ignore_errors=True)
shutil.rmtree(config.tmp_automl_path, ignore_errors=True)
automl = AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=60,
    tmp_folder=config.tmp_automl_path,  # automated
    output_folder=config.output_automl_path,
    # metric=roc_auc
)

automl.fit(X_train, 
            y_train,
            dataset_name='d2v_small')

############################################################################
# Print the final ensemble constructed by auto-sklearn
# ====================================================

print(automl.show_models())

[WARNING] [2021-01-02 16:38:23,085:AutoML(1):d2v_small] Capping the per_run_time_limit to 59.0 to have time for a least 2 models in each process.
[(1.000000, MyDummyClassifier(configuration=1, random_state=None)),
]


In [7]:
automl.score(X_test, y_test)
# automl.cv_results_

0.6343182840857957

In [8]:
from sklearn.metrics import *

###########################################################################
# Get the Score of the final ensemble
# ===================================

predictions = automl.predict(X_test)
proba = automl.predict_proba(X_test)


print("Accuracy score:", accuracy_score(y_test, predictions))
print("MCC:", matthews_corrcoef(y_test, predictions))

Accuracy score: 0.6343182840857957
MCC: 0.0


In [8]:
# Add to the recommender
recommender.set_classifier(automl)
# predict
recommender.predict(1,133)

NameError: name 'recommender' is not defined

In [ ]:
# # TODO
# def automl_binary_classifier(x):
#     return automl.predict_proba(x)[0][0]
# sklearn.metrics.plot_roc_curve(automl_binary_classifier, X_test, y_test)

In [ ]:
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=["Ignored","Matched"])
disp.plot()